In [ ]:
%%capture
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

**AT FIRST ADD DATA "badlad-new-buetcsefest2023"**

In [ ]:
%%capture
import sys, os, distutils.core
# Note: This is a faster way to install detectron2 in Colab, but it does not include all functionalities (e.g. compiled operators).
# See https://detectron2.readthedocs.io/tutorials/install.html for full installation instructions
!git clone 'https://github.com/facebookresearch/detectron2'
dist = distutils.core.run_setup("./detectron2/setup.py")
!python -m pip install {' '.join([f"'{x}'" for x in dist.install_requires])}
sys.path.insert(0, os.path.abspath('./detectron2'))

In [ ]:
# detectron2

from detectron2.utils.memory import retry_if_cuda_oom
from detectron2.utils.logger import setup_logger
from detectron2.checkpoint import DetectionCheckpointer
from detectron2.modeling import build_model
from detectron2.evaluation import COCOEvaluator, inference_on_dataset
import detectron2.data.transforms as T
from detectron2.data import detection_utils as utils
from detectron2.data import DatasetCatalog, MetadataCatalog, build_detection_test_loader, build_detection_train_loader, DatasetMapper
from detectron2.utils.visualizer import Visualizer
from detectron2.structures import BoxMode
from detectron2.engine import DefaultPredictor, DefaultTrainer
from detectron2.config import get_cfg
from detectron2 import model_zoo

import pandas as pd
import numpy as np
from tqdm.notebook import tqdm  # progress bar
import matplotlib.pyplot as plt
import json
import cv2
import copy
from typing import Optional

from IPython.display import FileLink

# torch
import torch

import gc

import warnings
# Ignore "future" warnings and Data-Frame-Slicing warnings.
warnings.filterwarnings('ignore')

setup_logger()

**HAVE TO INSTALL UNILM FROM DRIVE AS WE CHANGED THE META ARCHITECTURE OF IT**

In [1]:
!pip install gdown

!gdown 13aLElAwaHcwigsVERZ_iR6UVWtgWboWk

Downloading...
From (uriginal): https://drive.google.com/uc?id=13aLElAwaHcwigsVERZ_iR6UVWtgWboWk
From (redirected): https://drive.google.com/uc?id=13aLElAwaHcwigsVERZ_iR6UVWtgWboWk&confirm=t&uuid=2eaa13ee-3902-4dbc-a37e-9956a97631d9
To: /kaggle/working/unilm.zip
100%|█████████████████████████████████████████| 108M/108M [00:00<00:00, 180MB/s]


In [ ]:
# Replace '/kaggle/working/unilm.zip' with the actual path to your 'unilm.zip' file
zip_file_path = 'unilm.zip'

# Replace 'unilm' with the name of the folder where you want to unzip the contents
output_folder = 'unilm'


# Unzip the file
!unzip $zip_file_path -d $output_folder

In [ ]:
sys.path.insert(1, "/kaggle/working/unilm/layoutlmv3")


*****change to collections.abc due to newer version*****

In [ ]:
! sed -i 's/from collections import Iterable/from collections.abc import Iterable/' /kaggle/working/unilm/layoutlmv3/examples/object_detection/ditod/table_evaluation/data_structure.py

In [ ]:
!pip install timm
!pip install  shapely

In [ ]:
from examples.object_detection.ditod import add_vit_config

**test the config and meta architecture**

In [ ]:
cfg = get_cfg()
# Add PointRend-specific config
add_vit_config(cfg)
# Load a config from file
cfg.merge_from_file("unilm/layoutlmv3/examples/object_detection/cascade_layoutlmv3.yaml")
print(cfg)

In [ ]:
from pathlib import Path

TRAIN_IMG_DIR = Path("/kaggle/input/badlad-new-buetcsefest2023/train")

TRAIN_COCO_PATH = Path("/kaggle/input/badlad-new-buetcsefest2023/_annotations.coco (3).json")

# TEST_IMG_DIR = Path("badlad/images/test")

# TEST_METADATA_PATH = Path("badlad/badlad-test-metadata.json")

# Training output directory
OUTPUT_DIR = Path("./output_layoutlmv3")
OUTPUT_MODEL = OUTPUT_DIR/"model_final.pth"

# Path to your pretrained model weights
PRETRAINED_PATH = Path("")

In [ ]:
from datetime import datetime

# if False, model is set to `PRETRAINED_PATH` model
is_train = True

# if True, evaluate on validation dataset
is_evaluate = False

# if True, run inference on test dataset
is_inference = True

# if True and `is_train` == True, `PRETRAINED_PATH` model is trained further
is_resume_training = False

# Perform augmentation
is_augment = False

SEED = 42

In [ ]:
MODEL_PATH = OUTPUT_MODEL if is_train else PRETRAINED_PATH

In [ ]:
with TRAIN_COCO_PATH.open() as f:
    train_dict = json.load(f)

In [ ]:
def organize_coco_data(data_dict: dict) -> tuple[list[str], list[dict], list[dict]]:
    thing_classes: list[str] = []

    # Map Category Names to IDs
    for cat in data_dict['categories']:
        thing_classes.append(cat['name'])

    # Images
    images_metadata: list[dict] = data_dict['images']

    # Convert COCO annotations to detectron2 annotations format
    data_annotations = []
    for ann in data_dict['annotations']:
        # coco format -> detectron2 format
        annot_obj = {
            # Annotation ID
            "id": ann['id'],

            # Segmentation Polygon (x, y) coords
            "gt_masks": ann['segmentation'],

            # Image ID for this annotation (Which image does this annotation belong to?)
            "image_id": ann['image_id'],

            # Category Label (0: paragraph, 1: text box, 2: image, 3: table)
            "category_id": ann['category_id'],

            "x_min": ann['bbox'][0],  # left
            "y_min": ann['bbox'][1],  # top
            "x_max": ann['bbox'][0] + ann['bbox'][2],  # left+width
            "y_max": ann['bbox'][1] + ann['bbox'][3]  # top+height
        }
        data_annotations.append(annot_obj)

    return thing_classes, images_metadata, data_annotations

In [ ]:
thing_classes, images_metadata, data_annotations = organize_coco_data(
    train_dict
)

In [ ]:
thing_classes=["paragraph","text_box","image","table"]

In [ ]:
mapping = {
    2: 0,
    4: 1,
    1: 2,
    3: 3
}

In [ ]:
train_metadata = pd.DataFrame(images_metadata)
train_metadata = train_metadata[['id', 'file_name', 'width', 'height']]
train_metadata = train_metadata.rename(columns={"id": "image_id"})
print("train_metadata size=", len(train_metadata))
train_metadata.info(5)

In [ ]:
train_annot_df = pd.DataFrame(data_annotations)
train_annot_df['category_id'] = train_annot_df['category_id'].replace(mapping)
print("train_annot_df size=", len(train_annot_df))
train_annot_df.tail(20)

In [ ]:
TRAIN_SPLIT = 0.90

In [ ]:
n_dataset = len(train_metadata)
n_train = int(n_dataset * TRAIN_SPLIT)
print("n_dataset", n_dataset, "n_train", n_train, "n_val", n_dataset-n_train)

np.random.seed(SEED)

inds = np.random.permutation(n_dataset)
train_inds, valid_inds = inds[:n_train], inds[n_train:]

In [ ]:
DatasetCatalog.clear()

In [ ]:
DATA_REGISTER_TRAINING = "my_dataset_train"
DATA_REGISTER_VALID    = "my_dataset_val"

In [ ]:
def convert_coco_to_detectron2_format(
    imgdir: Path,
    metadata_df: pd.DataFrame,
    annot_df: Optional[pd.DataFrame] = None,
    target_indices: Optional[np.ndarray] = None,
):

    dataset_dicts = []
    for _, train_meta_row in tqdm(metadata_df.iterrows(), total=len(metadata_df)):
        # Iterate over each image
        image_id, filename, width, height = train_meta_row.values

        annotations = []

        # If train/validation data, then there will be annotations
        if annot_df is not None:
            for _, ann in annot_df.query("image_id == @image_id").iterrows():
                # Get annotations of current iteration's image
                class_id = ann["category_id"]
                gt_masks = ann["gt_masks"]
                bbox_resized = [
                    float(ann["x_min"]),
                    float(ann["y_min"]),
                    float(ann["x_max"]),
                    float(ann["y_max"]),
                ]

                annotation = {
                    "bbox": bbox_resized,
                    "bbox_mode": BoxMode.XYXY_ABS,
                    "segmentation": gt_masks,
                    "category_id": class_id,
                }

                annotations.append(annotation)

        # coco format -> detectron2 format dict
        record = {
            "file_name": str(imgdir/filename),
            "image_id": image_id,
            "width": width,
            "height": height,
            "annotations": annotations
        }

        dataset_dicts.append(record)

    if target_indices is not None:
        dataset_dicts = [dataset_dicts[i]  for i in target_indices]

    return dataset_dicts

In [ ]:
# Register Training data
DatasetCatalog.register(
    DATA_REGISTER_TRAINING,
    lambda: convert_coco_to_detectron2_format(
        TRAIN_IMG_DIR,
        train_metadata,
        train_annot_df,
        target_indices=train_inds,
    ),
)

# Set Training data categories
MetadataCatalog.get(DATA_REGISTER_TRAINING).set(thing_classes=thing_classes)

dataset_dicts_train = DatasetCatalog.get(DATA_REGISTER_TRAINING)
train_metadata1 = MetadataCatalog.get(DATA_REGISTER_TRAINING)

print("dicts training size=", len(dataset_dicts_train))
print("################")

In [ ]:
# Register Validation data
DatasetCatalog.register(
    DATA_REGISTER_VALID,
    lambda: convert_coco_to_detectron2_format(
        TRAIN_IMG_DIR,
        train_metadata,
        train_annot_df,
        
        target_indices=valid_inds,
    ),
)

# Set Validation data categories
MetadataCatalog.get(DATA_REGISTER_VALID).set(thing_classes=thing_classes)

dataset_dicts_valid = DatasetCatalog.get(DATA_REGISTER_VALID)
metadata_dicts_valid = MetadataCatalog.get(DATA_REGISTER_VALID)

print("dicts valid size=", len(dataset_dicts_valid))
print("################")

*HAVE TO DOWNLOAD config file of layoutlmv3 which will be available at

"https://huggingface.co/RadAlienware/layoutlmv3" which is now private due to the ongoing competition or pubicly available
https://huggingface.co/microsoft/layoutlmv3-base


Also load the pretrained weight from 
https://github.com/JPLeoRX/detectron2-publaynet/blob/master/mask_rcnn_R_50_FPN_3x/download.txt


All this can be fetched via gdown

In [2]:
!gdown 1CwIgwAFY4s7Nz-ST7Al2KGL1qtrlIhFx
!gdown 1IbxaRd82hIrxPT4a1U61_g2vvE3zcRLO

Downloading...
From: https://drive.google.com/uc?id=1CwIgwAFY4s7Nz-ST7Al2KGL1qtrlIhFx
To: /kaggle/working/config.json
100%|██████████████████████████████████████████| 897/897 [00:00<00:00, 5.57MB/s]
Downloading...
From (uriginal): https://drive.google.com/uc?id=1IbxaRd82hIrxPT4a1U61_g2vvE3zcRLO
From (redirected): https://drive.google.com/uc?id=1IbxaRd82hIrxPT4a1U61_g2vvE3zcRLO&confirm=t&uuid=1a1871c9-0c7f-48c4-8f51-dd817f1b5955
To: /kaggle/working/model_final.pth
100%|█████████████████████████████████████████| 176M/176M [00:00<00:00, 200MB/s]


In [ ]:
if is_train:  
    # cfg = AutoConfig.from_pretrained("model_final.pth")
    # cfg = AutoConfig.from_pretrained("layoutlmv3-base")
    add_vit_config(cfg)
    cfg.merge_from_file("unilm/layoutlmv3/examples/object_detection/cascade_layoutlmv3.yaml")
    cfg.MODEL.DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    cfg.MODEL.WEIGHTS = "/kaggle/working/model_final.pth"
    cfg.SOLVER.AMP.ENABLED = True
    BATCH_SIZE=3
    NOMINAL_BATCH_SIZE=27

    cfg.SOLVER.GAMMA = 0.0001
    cfg.MODEL.CONFIG_PATH="config.json"

    # Small value == Frequent save need a lot of storage.
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
    cfg.MODEL.ROI_HEADS.NUM_CLASSES = 4
    cfg.SOLVER.BASE_LR = 0.001 * BATCH_SIZE/NOMINAL_BATCH_SIZE
    cfg.SOLVER.IMS_PER_BATCH = BATCH_SIZE
    cfg.SOLVER.WARMUP_ITERS = 100
    cfg.DATALOADER.NUM_WORKERS = 2
    cfg.SOLVER.MAX_ITER = 92000 # Set higher value, like 10000 for better results
    cfg.MODEL.ROI_HEADS.BATCH_SIZE_PER_IMAGE = 128
    cfg.SOLVER.CHECKPOINT_PERIOD = 1000
    cfg.DATASETS.TRAIN = (DATA_REGISTER_TRAINING,)
    # cfg.DATASETS.TEST=(DATA_REGISTER_TEST,)
    cfg.SOLVER.GRADIENT_ACCUMULATION_STEPS = 1
    cfg.OUTPUT_DIR = str(OUTPUT_DIR)
    
    print("creating cfg.OUTPUT_DIR -> ", cfg.OUTPUT_DIR)
    OUTPUT_DIR.mkdir(exist_ok=True)

In [ ]:
from examples.object_detection.ditod import MyTrainer

# os.makedirs(cfg.OUTPUT_DIR, exist_ok=True)
trainer = MyTrainer(cfg) 
trainer.resume_or_load(resume=True)
# Uncomment to train
trainer.train()